# Web server for the MarsCamBot remote control

In [1]:
# Import the Time library
import time  
# Import the GPIO Zero Library
import gpiozero 

# For receiving images from the camera
import cv2

# For the Web server
import flask  

In [2]:
# Initialise right and left motors
right_motor = gpiozero.Motor(9, 10)
left_motor = gpiozero.Motor(7, 8)

# Set motor speed
motor_speed=0.5

In [3]:
# Function to go forward
def forward(speed=0.5, duration=1):

    # Forward: Turn both motors on with the speed "speed"
    left_motor.forward(speed)
    right_motor.forward(speed)

    # Wait for "duration" seconds
    time.sleep(duration)

    # Turn the motors off
    left_motor.stop()
    right_motor.stop()

# Function to go backward
def backward(speed=0.5, duration=1):

    # Backward: Turn both motors on with the speed "-speed"
    left_motor.backward(speed)
    right_motor.backward(speed)

    # Wait for "duration" seconds
    time.sleep(duration)

    # Turn the motors off
    left_motor.stop()
    right_motor.stop()

# Function to turn right    
def turn_right(speed=0.5, duration=1):

    # right: Only turn on left motor with a speed of 0.5 
    right_motor.backward()
    left_motor.forward(speed)

    # Wait for "duration" seconds
    time.sleep(duration)

    # Turn the motors off
    left_motor.stop()
    right_motor.stop()
    
# Function to turn left    
def turn_left(speed=0.5, duration=1):

    # left: Only turn on left motor with a speed of 0.5 
    right_motor.forward(speed)
    left_motor.backward(speed)

    # Wait for "duration" seconds
    time.sleep(duration)

    # Turn the motors off
    left_motor.stop()
    right_motor.stop()
    

In [4]:
# Display the video
def display_video():
    
    camera_object = cv2.VideoCapture(0)
    
    while True:
    
        ret, picture = camera_object.read()
        picture_rgb = cv2.cvtColor(picture, cv2.COLOR_BGR2RGB)
    
        ret, jpeg = cv2.imencode('.jpg', picture_rgb)
        pic = jpeg.tobytes()
        
        #Flask streaming
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + pic + b'\r\n\r\n')

In [5]:
# Start Flask app (Web server)
app = flask.Flask("Remote control")

In [6]:
# Define the control as a function of the URL
@app.route('/')
def index():
    return flask.render_template("index.html")

@app.route('/video_feed')
def video_feed():
    return flask.Response(display_video(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/forward')
def button_forward():
    forward(speed=motor_speed, duration=1)
    return ""
    
@app.route('/backward')
def button_backward():
    backward(speed=motor_speed, duration=1)
    return ""
    
@app.route('/left')
def button_left():
    turn_left(speed=motor_speed, duration=1)
    return ""
    
@app.route('/right')
def button_right():
    turn_right(speed=motor_speed, duration=1)
    return ""


In [7]:
# Start Web server
app.run(host='0.0.0.0', port=2204, threaded=True, debug=False)

 * Serving Flask app "Remote control" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:2204/ (Press CTRL+C to quit)
127.0.0.1 - - [16/May/2022 09:58:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 09:58:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [16/May/2022 09:59:00] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 09:59:03] "GET /forward HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 10:07:11] "GET /forward HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 10:07:14] "GET /backward HTTP/1.1" 200 -
